In [5]:
import os
all_txt_files =[]
# get all file names for obits in txt folder
for root, dirs, files in os.walk("txt"):
    for file in files:
        if file.endswith(".txt"):
            all_txt_files.append(os.path.join(root, file))
n_files = len(all_txt_files)
all_txt_files[365]

'txt/0426.txt'

In [6]:
# create a list of strings. Each string is an obit
all_text = []
for i in all_txt_files:
    with open(i) as f:
        txt = f.read()
    all_text.append(txt)

In [7]:
# import modules
import gensim
import multiprocessing

# check number of CPUs
multiprocessing.cpu_count()

8

In [13]:
import spacy
nlp = spacy.load('en')

# define a function to tokenize, lowercase all, remove punctuation, remove numbers etc.
def text_cleanup(mystring):
    x = ['SPACE', 'PUNCT', 'SYM', 'X', 'NUM']
    doc = nlp(mystring.lower())
    processed = [i for i in doc if i.pos_ not in x]
    processed = [i.text for i in processed if i.is_stop != True]
    return processed

In [27]:
# this code will take a long time to execute ... 
all_docs = [text_cleanup(i) for i in all_text]

In [30]:
model = gensim.models.Word2Vec(all_docs, size=500, window=6, min_count=2, workers=8)

2018-11-15 17:29:56,518 : INFO : collecting all words and their counts
2018-11-15 17:29:56,519 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-11-15 17:29:56,597 : INFO : collected 35901 word types from a corpus of 454105 raw words and 366 sentences
2018-11-15 17:29:56,597 : INFO : Loading a fresh vocabulary
2018-11-15 17:29:56,642 : INFO : min_count=2 retains 21801 unique words (60% of original 35901, drops 14100)
2018-11-15 17:29:56,643 : INFO : min_count=2 leaves 440005 word corpus (96% of original 454105, drops 14100)
2018-11-15 17:29:56,725 : INFO : deleting the raw counts dictionary of 35901 items
2018-11-15 17:29:56,727 : INFO : sample=0.001 downsamples 12 most-common words
2018-11-15 17:29:56,727 : INFO : downsampling leaves estimated 426319 word corpus (96.9% of prior 440005)
2018-11-15 17:29:56,776 : INFO : estimated required memory for 21801 words and 500 dimensions: 98104500 bytes
2018-11-15 17:29:56,777 : INFO : resetting layer weights
2018-

In [32]:
model.most_similar("love")

/Users/lavin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('girl', 0.9990344047546387),
 ('write', 0.998995840549469),
 ('unrequited', 0.9988707304000854),
 ('friends', 0.9986531138420105),
 ("'re", 0.9984126091003418),
 ('read', 0.9983095526695251),
 ('pickford', 0.9982596635818481),
 ('nightingale', 0.9982486367225647),
 ('recalled', 0.9981932044029236),
 ('live', 0.9981801509857178)]

In [33]:
model.similarity('woman', 'man')

/Users/lavin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.8615039758465234

In [34]:
model.similarity('success', 'wealth')

/Users/lavin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """Entry point for launching an IPython kernel.


0.995969269476518

In [35]:
pairs = []
for y in model.wv.vocab.keys():
    score = model.similarity('wealth', y)
    pairs.append((y, score))

/Users/lavin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [36]:
import pandas as pd 
df = pd.DataFrame.from_records(pairs, columns=["word", "score"]).sort_values(by="score", ascending=False)
df

,word,score
6872,wealth,1.000000
605,fingers,0.999902
2283,aware,0.999880
663,scenes,0.999845
925,murder,0.999835
5875,riding,0.999828
11578,bands,0.999827
7833,admirers,0.999821
1262,cast,0.999817
14074,uncommon,0.999816
